In [1]:
import json

import pandas as pd

base_path_results: str = '/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models'

base_model_llama_8B: str = 'Unsloth-meta-llama-3.1-4bit-plain-t0.0--Unsloth-meta-llama-3.1-4bit-plain-t0.0'
final_model_llama_8B: str = 'llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0'
base_model_llama_70B: str = 'Unsloth-meta-llama-3.1-70B-bnb-4bit-t0.0--Unsloth-meta-llama-3.1-70B-bnb-4bit-t0.0'
base_model_metal_llama_70B: str = 'Meta-Llama-3.1-70B-Instruct-t0.0--Meta-Llama-3.1-70B-Instruct-t0.0'


final_model_llama_70B: str = 'llama3.1-70B-sft-e1-DFINAL_0.6K-steps-t0.0--llama3.1-70B-sft-e1-DFINAL_0.6K-steps-t0.0'
base_model_mistral: str = 'Unsloth-Mistral-Small-24B-Instruct-2501-t0.0--Unsloth-Mistral-Small-24B-Instruct-2501-t0.0'
final_model_mistral: str = 'Mistral-small-2501-DFINAL_0.6K-steps-t0.0--Mistral-small-2501-DFINAL_0.6K-steps-t0.0'

std_cols: list = ['game', 'model', 'experiment', 'episode']
model_files: list[str] = ['llama_8B', 'llama_70B', 'mistral']

current_base_model = base_model_mistral
current_final_model = final_model_mistral
model_file = 'mistral'
current_game = "matchit_ascii"
raw_csv = "raw.csv"

In [21]:
column_order_adventuregame = [
    'Success',
    'Lose',
    'Aborted',
    'Main Score',
    'Played',
    'achieved_goal_ratio',
    'hallucination_count',
    'malformed_command',
    'undefined_repr_str',
    'undefined_action_verb',
    'action_parsing_fail',
    'Violated Request Count',
    'lark_exception',
    #'turn_limit_loss',
    'turn_ratio',
    'turns_over_par',
    'finish_speed',
    'entity_not_accessible',
    'entity_already_inventory',
    'action_resolution_fail',
    'inventory_limit_exceeded',
    #'going_to_current_room',
    'no_exit_to',
    'other_room_argument',
    #'world_state_discrepancy',
    #'domain_type_discrepancy',
    'bad_plan_dismiss_ratio',
    'bad_plan_follow_ratio',
    'plan_average_viability_ratio',
    'plan_followed_ratio',
    #'successful_actions',
    #'undefined_action',
    #'undefined_argument_type',
    #'Request Success Ratio',
    #'entity_state_mismatch',
    #'entity_trait_mismatch',
    #'taking_from_inventory',
   # 'domain_trait_type_mismatch',
    #'Request Count',
    #'Parsed Request Count',
    #'manipulating_room',
]
sub_headers_adventuregame = {
    "Success": "Game Outcome",
    "turn_ratio": "Efficiency",
    "hallucination_count": "Instruction Following \\& Output Formatting",
    "tun_limit_loss": "Efficiency (When Success/ Loss)",
    "entity_not_accessible": "Context Awareness \\& Memory",
    "bad_plan_dismiss_ratio": "Planning",
    # "successful_actions": "Action Selection \\& Execution",
    #"entity_state_mismatch": "Situational Understanding \\& Common Sense",
}
metrics_config_adventuregame = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'achieved_goal_ratio': True,
    'Played': True,
    'hallucination_count': False,
    'malformed_command': False,
    'undefined_repr_str': False,
    'Violated Request Count': False,
    'lark_exception': False,
    'turn_limit_loss': False,
    'turn_ratio': False,
    'turns_over_par': False,
    'finish_speed': True,
    'entity_not_accessible': False,
    'entity_already_inventory': False,
    'going_to_current_room': False,
    'no_exit_to': False,
    'other_room_argument': False,
    'world_state_discrepancy': False,
    'domain_type_discrepancy': False,
    'bad_plan_dismiss_ratio': False,
    'bad_plan_follow_ratio': False,
    'plan_average_viability_ratio': True,
    'plan_followed_ratio': True,
    'successful_actions': True,
    'undefined_action': False,
    'undefined_action_verb': False,
    'undefined_argument_type': False,
    'action_parsing_fail': False,
    'action_resolution_fail': False,
    'Request Success Ratio': True,
    'entity_state_mismatch': False,
    'entity_trait_mismatch': False,
    'taking_from_inventory': False,
    'domain_trait_type_mismatch': False,
    'inventory_limit_exceeded': False,
    'Request Count' : True,
    'Parsed Request Count' : True,
    'manipulating_room': False,
}
metrics_when_success_adventuregame = [
    'turn_ratio',
    'turns_over_par',
    'finish_speed',
]

column_order_tmw_graph = ['Success',
                          'Lose',
                          'Aborted',
                          'Main Score',
                          'Played',

                          'efficiency',
                          'loops',
                          'moves',

                          'exploration',
                          'number_visited',
                          'seen',
                          ]
sub_headers_tmw_graph = {
    "Success": "Game Outcome",
    "efficiency": "Efficiency",
    "exploration": "Exploration",
}
metrics_config_tmw_graph = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'efficiency': True,
    'exploration': True,
    'loops': False,
    'valid_moves': True,
    "seen": True,
    "invalid_moves": True,
    "moves": False
}
metrics_when_success_tmw_graph = [
]

column_order_tmw = ['Success',
                      'Lose',
                      'Aborted',
                      'Main Score',
                      'Played',

                    'efficiency',
                      'loops',
                      'moves',

                    'exploration',
                      'number_visited',
                      'seen',
                    ]
sub_headers_tmw = {
    "Success": "Game Outcome",
    "efficiency": "Efficiency",
    "exploration": "Exploration",
}
metrics_config_tmw = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'efficiency': True,
    'exploration': True,
    'loops': False,
    'valid_moves': True,
       "moves": False,
    "seen": True,
    'number_visited': True
}
metrics_when_success_tmw = []

column_order_tmw_specificroom = ['Success',
                      'Lose',
                      'Aborted',
                      'Main Score',
                      'Played',

                    'efficiency',
                      'loops',
                      'moves',

                    'exploration',
                      'number_visited',
                      'seen',]
sub_headers_tmw_specificroom  = {
    "Success": "Game Outcome",
    "efficiency": "Efficiency",
    "exploration": "Exploration",
}
metrics_config_tmw_specificroom  = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'efficiency': True,
    'exploration': True,
    'loops': False,
    'valid_moves': True,
    "moves": False,
    "seen": True,
    'number_visited': True
}
metrics_when_success_tmw_specificroom = []

column_order_codenames = [
                         'Success',
                          'Lose',
                          'Aborted',
                          'Main Score',
                          'Played',
                          'Game ended through assassin',

                        'Efficiency',
                        'Number of turns',
                        'Episode Recall',

                        'Average Cluegiver Number of Targets',
                        'Average Cluegiver Team F1',

                        'Average Guesser Number of Guesses',
                        'Average Guesser Number of Revealed Words',
                        'Average Guesser Number of Unrevealed Guesses',
                        'Average Guesser Target F1',
]
sub_headers_codenames = {
    "Success": "Game Outcome",
    "Efficiency": "Efficiency",
    "Average Cluegiver Number of Targets" : "Individual Cluegiver Metrics",
    "Average Guesser Number of Guesses": "Individual Guesser Metrics",
    "Violated Request Count": "Instruction Following"

}
metrics_config_codenames = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Efficiency' : True,
    'Episode Negative Recall': False,
    'Episode Recall' : True,
    'Game ended through assassin': False,
    'Number of turns': False,
    'Parsed Request Count': False,
    'Request Count': False,
    'Request Success Ratio': True,
    'Violated Request Count': False,
    'Average Cluegiver Number of Targets' : True,
    'Average Cluegiver Team F1': True,
    'Average Cluegiver Team Precision' : True,
    'Average Cluegiver Team Recall': True,
    'Average Guesser Number of Guesses': True,
    'Average Guesser Number of Revealed Words': True,
    'Average Guesser Number of Unrevealed Guesses': False,
    'Average Guesser Target F1': True,
    'Average Guesser Target Precision': True,
    'Average Guesser Target Recall': True,
    'Average Guesser Team F1': True,
    'Average Guesser Team Precision': True,
    'Average Guesser Team Recall': True,
}
metrics_when_success_codenames = []

column_order_guesswhat = [ 'Success',
                           'Lose',
                           'Aborted',
                           'Main Score',
                           'Played',

                           'Speed',
                           'Request Count',
                           ]
sub_headers_guesswhat = {
    'Success': "Game Outcome",
    "Speed": "Efficiency",
    'Invalid content guesser response': "Player Individual"
}
metrics_config_guesswhat = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Invalid content answerer response': False,
    'Invalid content guesser response': False,
    'Invalid format answerer response': False,
    'Invalid format guesser response': False,
    'Parsed Request Count': True,
    'Request Count': False,
    'Request Success Ratio' : True,
    'Speed' : True,
    'Violated Request Count': False
}
metrics_when_success_guesswhat = []

column_order_imagegame = [ 'Success',
                           'Lose',
                           'Aborted',
                           'Main Score',
                           'Played',
                           'Average Changed Cell Count',
                           'Average Generated Instruction Length',
                           'Request Count',
                           'F1',
                           'Precision',
                           'Recall',

                           ]
sub_headers_imagegame = {
    'Success': "Game Outcome",
    'Average Changed Cell Count': "Efficiency",
    'F1': 'F-Measure'
}
metrics_config_imagegame = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Average Changed Cell Count': True,
    'F1': True,
    'Parsed Request Count' : True,
    'Precision' : True,
    'Recall': True,
    'Request Success Ratio': True,
    }
metrics_when_success_imagegame = [
]

column_order_privateshared = ['Success',
                              'Lose',
                              'Aborted',
                              'Main Score',
                              'Played',
                              'Accuracy',
                              'Middle-Accuracy',
                              'Slot-Filling-Accuracy',
                              ]
sub_headers_privateshared = {
        "Success": "Game Outcome",
        'Accuracy': "Accuracy"
}
metrics_config_privateshared = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Accuracy': True,
    'Kappa': True,
    'Middle-Accuracy': True,
    'Parsed Request Count': True,
    'Request Success Ratio': True,
    'Slot-Filling-Accuracy': True,
    'Truncated Kappa': True,
    'Violated Request Count': False
    }
metrics_when_success_privateshared = []

column_order_referencegame = ['Success',
                              'Lose',
                              'Aborted',
                              'Main Score',
                              'Played',

                              'Generated Expression Length',
                              'Request Count',
                              ]
sub_headers_referencegame = {
    "Success": "Game Outcome",
    'Generated Expression Length': "Efficiency"
}
metrics_config_referencegame = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Aborted at Player 1': False,
    'Aborted at Player 2': False,
    'Parsed Request Count': True,
    'Request Success Ratio': True,
    'Violated Request Count': False
    }
metrics_when_success_referencegame = [
    'Generated Expression Length',
    'Request Count',
]

column_order_taboo = ['Success',
                      'Lose',
                      'Aborted',
                      'Main Score',
                      'Played',
                      'Request Count',
                      'Repetition-Describer',
                      'Repetition-Guesser',

                      ]
sub_headers_taboo = {
    "Success": "Game Outcome",
    'Request Count': "Efficiency"
}
metrics_config_taboo = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Parsed Request Count': True,
    'Repetition-Describer': False,
    'Repetition-Guesser': False,
    'Request Success Ratio': True,
    'Violated Request Count': False
}
metrics_when_success_taboo = [
    'Request Count',
]

column_order_wordle = ['Success',
                       'Lose',
                       'Aborted',
                       'Main Score',
                       'Played',

                       'Request Count',
                       'total guess repetitions'
                       ]
sub_headers_wordle = {
    "Success": "Game Outcome",
    "Request Count": "Efficiency"
}
metrics_config_wordle = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Parsed Request Count': True,
    'Request Success Ratio': True,
    'Violated Request Count': False,
    'repeats guess': False,
    'total guess repetitions': False
    }
metrics_when_success_wordle = []

column_order_wordle_withclue = ['Success',
                               'Lose',
                               'Aborted',
                               'Main Score',
                               'Played',

                               'Request Count',
                               'total guess repetitions']
sub_headers_wordle_withclue = {
    "Success": "Game Outcome",
    "Request Count": "Efficiency"
}

metrics_config_wordle_withclue = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Parsed Request Count': True,
    'Request Success Ratio': True,
    'Violated Request Count': False,
    'repeats guess': False,
    'total guess repetitions': False,
    }
metrics_when_success_wordle_withclue = []

column_order_wordle_withcritic = ['Success',
                                  'Lose',
                                  'Aborted',
                                  'Main Score',
                                  'Played',

                                  'Request Count',
                                  'total guess repetitions',

                                  'Non-Repetition-Guesser-On-Critic-Agreement',
                                  'Non-Repetition-Guesser-On-Critic-Disagreement',
                                  'Repetition-Guesser-On-Critic-Agreement',
                                  'Repetition-Guesser-On-Critic-Disagreement',
                                  ]
sub_headers_wordle_withcritic = {
    "Success": "Game Outcome",
    "Request Count": "Efficiency",
    "Non-Repetition-Guesser-On-Critic-Agreement":  "Guesser Reaction on Critic"
}
metrics_config_wordle_withcritic = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Non-Repetition-Guesser-On-Critic-Agreement': True,
    'Non-Repetition-Guesser-On-Critic-Disagreement': True,
    'Parsed Request Count': True,
    'Repetition-Guesser-On-Critic-Agreement': False,
    'Repetition-Guesser-On-Critic-Disagreement': False,
    'Request Success Ratio': True,
    'Violated Request Count': False,
    'repeats guess': False,
    'total guess repetitions': False
}
metrics_when_success_wordle_withcritic= []


column_order_matchit_ascii = [  'Success',
                                'Lose',
                                'Aborted',
                                'Main Score',
                                'Played',
                                'Player Score',
                            ]
sub_headers_matchit_ascii = {
    "Success": "Game Outcome",
}
metrics_config_matchit_ascii = {
    'Main Score': True,
    'Success': True,
    'Lose': False,
    'Aborted': False,
    'Played': True,
    'Parsed Request Count': True,
    'Player Score': True,
    'Violated Request Count': False
}
metrics_when_success_matchit_ascii = []






metric_orders = {
    "imagegame": column_order_imagegame,
    "privateshared": column_order_privateshared,
    "referencegame": column_order_referencegame,
    "taboo": column_order_taboo,
    "wordle": column_order_wordle,
    "wordle_withclue": column_order_wordle_withclue,
    "wordle_withcritic": column_order_wordle_withcritic,
    "adventuregame": column_order_adventuregame,
    "codenames": column_order_codenames,
    "textmapworld": column_order_tmw,
    "textmapworld_specificroom": column_order_tmw_specificroom,
    "textmapworld_graphreasoning": column_order_tmw_graph,
    "guesswhat": column_order_guesswhat,
    "matchit_ascii": column_order_matchit_ascii
}
all_sub_headers = {
    "imagegame": sub_headers_imagegame,
    "privateshared": sub_headers_privateshared,
    "referencegame": sub_headers_referencegame,
    "taboo": sub_headers_taboo,
    "wordle": sub_headers_wordle,
    "wordle_withclue": sub_headers_wordle_withclue,
    "wordle_withcritic": sub_headers_wordle_withcritic,
    "adventuregame": sub_headers_adventuregame,
    "codenames": sub_headers_codenames,
    "textmapworld": sub_headers_tmw,
    "textmapworld_specificroom": sub_headers_tmw_specificroom,
    "textmapworld_graphreasoning": sub_headers_tmw_graph,
    "guesswhat": sub_headers_guesswhat,
    "matchit_ascii": sub_headers_matchit_ascii
}
metrics_configs = {
    "imagegame": metrics_config_imagegame,
    "privateshared": metrics_config_privateshared,
    "referencegame": metrics_config_referencegame,
    "taboo": metrics_config_taboo,
    "wordle": metrics_config_wordle,
    "wordle_withclue": metrics_config_wordle_withclue,
    "wordle_withcritic": metrics_config_wordle_withcritic,
    "adventuregame": metrics_config_adventuregame,
    "codenames": metrics_config_codenames,
    "textmapworld": metrics_config_tmw,
    "textmapworld_specificroom": metrics_config_tmw_specificroom,
    "textmapworld_graphreasoning": metrics_config_tmw_graph,
    "guesswhat": metrics_config_guesswhat,
    "matchit_ascii": metrics_config_matchit_ascii
}
model_lookup = {
    'llama_8B_base': 'llama_8B_fine',
    'mistral_base': 'mistral_fine',
    'llama_70B_base': 'llama_70B_fine'
}
success_only_cols = {
    "imagegame": metrics_when_success_imagegame,
    "privateshared": metrics_when_success_privateshared,
    "referencegame": metrics_when_success_referencegame,
    "taboo": metrics_when_success_taboo,
    "wordle": metrics_when_success_wordle,
    "wordle_withclue": metrics_when_success_wordle_withclue,
    "wordle_withcritic": metrics_when_success_wordle_withcritic,
    "adventuregame": metrics_when_success_adventuregame,
    "codenames": metrics_when_success_codenames,
    "textmapworld": metrics_when_success_tmw,
    "textmapworld_specificroom": metrics_when_success_tmw_specificroom,
    "textmapworld_graphreasoning": metrics_when_success_tmw_graph,
    "guesswhat": metrics_when_success_guesswhat,
    "matchit_ascii": metrics_when_success_matchit_ascii
}

general_column_alias = {
    'Main Score': "Quality Score",
}

base_metrics =[
    'Success',
    'Lose',
    'Aborted',
    'Played',
]

In [22]:
import math


def load_raw_csv(mode_file: str, base_model: str, final_model: str, game: str) -> pd.DataFrame:
    path: str = base_path_results + '/' + mode_file + '/' + raw_csv
    df = pd.read_csv(path)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df.metric = df.metric.astype('string')
    df.value = df.value.astype('string')
    df = df[df['game'] == game]
    metrics_for_game_df = df[df.game == game]
    unique_metrics: list = list(metrics_for_game_df['metric'].unique())
    df_pivoted = group_raw_csv(df, base_model, final_model)

    return reduce_to_metrics_for_game(df_pivoted, unique_metrics)

def group_raw_csv(df: pd.DataFrame, bm, fm) -> pd.DataFrame:
    df_pivoted = df.pivot(index=['game', 'model', 'experiment', 'episode'],
                          columns='metric',
                          values='value')

    # Reset the index to make the index columns regular columns
    df_pivoted = df_pivoted.reset_index()
    return df_pivoted

def reduce_to_metrics_for_game(df: pd.DataFrame, columns_to_keep) -> pd.DataFrame:
    std_cols: list = ['game', 'model', 'experiment', 'episode']
    columns_to_keep = std_cols + columns_to_keep
    columns_to_drop = [ c for c in list(df.columns) if c not in columns_to_keep]
    return df.drop(columns=columns_to_drop, axis=1)

def prepare_result_df(current_game: str, metric_order: str, all_success_metrics: list = [], base_metrics: list = []) -> (pd.DataFrame, ):

    df_mistral_raw = load_raw_csv("mistral" ,"", "", current_game)
    df_llama8B_raw = load_raw_csv("llama_8B" ,"", "", current_game)
    df_llama70B_raw = load_raw_csv("llama_70B"  ,"", "", current_game)


    for col in df_llama8B_raw.columns:
        if col not in std_cols:
            df_mistral_raw[col] = pd.to_numeric(df_mistral_raw[col], errors='coerce')
            df_llama8B_raw[col] = pd.to_numeric(df_llama8B_raw[col], errors='coerce')
            df_llama70B_raw[col] = pd.to_numeric(df_llama70B_raw[col], errors='coerce')

    print(df_llama8B_raw.columns)

    # This is the base data with success lost and aborted
    df_base_model_mistral = df_mistral_raw[df_mistral_raw.model == base_model_mistral].drop(['game', 'model','episode', 'experiment'], axis=1).fillna(0)
    df_final_model_mistral = df_mistral_raw[df_mistral_raw.model == final_model_mistral].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)

    df_base_model_llama8B = df_llama8B_raw[df_llama8B_raw.model == base_model_llama_8B].drop(['game', 'model','episode', 'experiment'], axis=1).fillna(0)
    df_final_model_llama8B = df_llama8B_raw[df_llama8B_raw.model == final_model_llama_8B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)

    df_base_model_llama70B = df_llama70B_raw[df_llama70B_raw.model == base_model_llama_70B].drop(['game', 'model','episode', 'experiment'], axis=1).fillna(0)
    df_final_model_llama70B = df_llama70B_raw[df_llama70B_raw.model == final_model_llama_70B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)

    # prepare the played data that only contains success and lost
    df_base_model_mistral_played = df_mistral_raw[(df_mistral_raw.model == base_model_mistral) & (df_mistral_raw.Aborted != 1)].drop(['game', 'model','episode', 'experiment'], axis=1).fillna(0)
    df_final_model_mistral_played = df_mistral_raw[(df_mistral_raw.model == final_model_mistral) & (df_mistral_raw.Aborted != 1)].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)

    df_base_model_llama8B_played = df_llama8B_raw[(df_llama8B_raw.model == base_model_llama_8B) & (df_llama8B_raw.Aborted != 1)].drop(['game', 'model','episode', 'experiment'], axis=1).fillna(0)
    df_final_model_llama8B_played = df_llama8B_raw[(df_llama8B_raw.model == final_model_llama_8B) & (df_llama8B_raw.Aborted != 1)].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)

    df_base_model_llama70B_played = df_llama70B_raw[(df_llama70B_raw.model == base_model_llama_70B)  & (df_llama70B_raw.Aborted != 1)].drop(['game', 'model','episode', 'experiment'], axis=1).fillna(0)
    df_final_model_llama70B_played = df_llama70B_raw[(df_llama70B_raw.model == final_model_llama_70B) & (df_llama70B_raw.Aborted != 1)].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)

    ## prepare all success lost and abroted df
    df_l8_base = df_llama8B_raw[df_llama8B_raw.model == base_model_llama_8B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
    df_l8_base_all_success = df_l8_base[df_l8_base.Success == 1]

    df_l8_final = df_llama8B_raw[df_llama8B_raw.model == final_model_llama_8B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
    df_l8_fine_all_success = df_l8_final[df_l8_final.Success == 1]

    df_l70_base = df_llama70B_raw[df_llama70B_raw.model == base_model_llama_70B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
    df_l70_base_all_success = df_l70_base[df_l70_base.Success == 1]

    df_l70_final = df_llama70B_raw[df_llama70B_raw.model == final_model_llama_70B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
    df_l70_fine_all_success = df_l70_final[df_l70_final.Success == 1]

    df_mistral_base = df_mistral_raw[df_mistral_raw.model == base_model_mistral].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
    df_mistral_base_all_success = df_mistral_base[df_mistral_base.Success == 1]

    df_mistral_final = df_mistral_raw[df_mistral_raw.model == final_model_mistral].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
    df_mistral_fine_all_success = df_mistral_final[df_mistral_final.Success == 1]

    results = {
        "metric" : metric_order,
        "llama_8B_base" : [],
        "llama_8B_fine" : [],
        "mistral_base": [],
        "mistral_fine" : [],
        "llama_70B_base" : [],
        "llama_70B_fine" : [],
    }

    for metric in metric_order:
        if metric in all_success_metrics:
            base_model_llama_8B_metrics = list(df_l8_base_all_success[metric])
            final_model_llama_8B_metrics = list(df_l8_fine_all_success[metric])

            base_model_llama_70B_metrics = list(df_l70_base_all_success[metric])
            final_model_llama_70B_metrics = list(df_l70_fine_all_success[metric])

            base_model_mistral_metrics = list(df_mistral_base_all_success[metric])
            final_model_mistral_metrics = list(df_mistral_fine_all_success[metric])
        elif metric not in  base_metrics:
            base_model_llama_8B_metrics = list(df_base_model_llama8B_played[metric])
            final_model_llama_8B_metrics = list(df_final_model_llama8B_played[metric])

            base_model_llama_70B_metrics = list(df_base_model_llama70B_played[metric])
            final_model_llama_70B_metrics = list(df_final_model_llama70B_played[metric])

            base_model_mistral_metrics = list(df_base_model_mistral_played[metric])
            final_model_mistral_metrics = list(df_final_model_mistral_played[metric])
        else:
            base_model_llama_8B_metrics = list(df_base_model_llama8B[metric])
            final_model_llama_8B_metrics = list(df_final_model_llama8B[metric])

            base_model_llama_70B_metrics = list(df_base_model_llama70B[metric])
            final_model_llama_70B_metrics = list(df_final_model_llama70B[metric])

            base_model_mistral_metrics = list(df_base_model_mistral[metric])
            final_model_mistral_metrics = list(df_final_model_mistral[metric])

        results["llama_8B_base"].append(sum(base_model_llama_8B_metrics)/len(base_model_llama_8B_metrics) if len(base_model_llama_8B_metrics) != 0 else 0)
        results["llama_8B_fine"].append(sum(final_model_llama_8B_metrics)/len(final_model_llama_8B_metrics) if len(final_model_llama_8B_metrics) != 0 else 0)

        results["llama_70B_base"].append(sum(base_model_llama_70B_metrics)/len(base_model_llama_70B_metrics) if len(base_model_llama_70B_metrics) != 0 else 0)
        results["llama_70B_fine"].append(sum(final_model_llama_70B_metrics)/len(final_model_llama_70B_metrics) if len(final_model_llama_70B_metrics) != 0 else 0)

        results["mistral_base"].append(sum(base_model_mistral_metrics)/len(base_model_mistral_metrics) if len(base_model_mistral_metrics) != 0 else 0)
        results["mistral_fine"].append(sum(final_model_mistral_metrics)/len(final_model_mistral_metrics) if len(final_model_mistral_metrics) != 0 else 0)

    result_df = pd.DataFrame(results)
    return result_df, {
        "llama_8B_base" : df_base_model_llama8B,
        "llama_8B_fine" : df_final_model_llama8B,
        "mistral_base": df_base_model_mistral,
        "mistral_fine" : df_final_model_mistral,
        "llama_70B_base" : df_base_model_llama70B,
        "llama_70B_fine" : df_final_model_llama70B,
        }

def prepare_metric_name(metric: str, metric_alias: dict = {}) -> str:
    if metric in metric_alias.keys():
        return metric_alias[metric]
    metric = metric.replace("_", " ")
    parts = metric.split(" ")
    parts = [p.capitalize() for p in parts]

    return " ".join(parts)

def prepare_row(row: dict, models, metric_config: dict, min_value: int, max_value: int, success_only_cols: list, min_positive_col: str = "", max_positive_col:str = "", min_negative_color: str= "", max_negative_color:str = "", metric_alias: dict = {}):
    metric = f'{prepare_metric_name(row["metric"], metric_alias=metric_alias)} (when success)' if row["metric"] in success_only_cols else prepare_metric_name(row["metric"], metric_alias=metric_alias)
    print(success_only_cols)
    str_row = metric.replace("_", "\\_")
    base_model = ""
    if row["metric"] in metric_config.keys():
        higher_better = metric_config[row["metric"]]
        for i, model in enumerate(models):
            if i % 2 == 0:
                base_model = model
            if model == base_model:
                str_row += " & " + f"{row[model]:.2f}"
            else:
                diff = row[model] - row[base_model]
                if higher_better and diff > 0:
                    str_row += " & " + "{\\color[HTML]{" + get_color(math.fabs(diff), 0 , math.fabs(max_value), min_positive_col, max_positive_col,) + "}" f" {math.fabs(diff):.2f} " + "}"
                elif not higher_better and diff > 0:
                    str_row += " & " + "{\\color[HTML]{" +  get_color(math.fabs(diff), 0, math.fabs(min_value), min_negative_color, max_negative_color,) + "}" f" {math.fabs(diff):.2f} " + "}"
                elif higher_better and diff < 0:
                    str_row += " & " + "{\\color[HTML]{" +  get_color(math.fabs(diff), 0, math.fabs(min_value), min_negative_color, max_negative_color,) + "}" f" {diff:.2f} " + "}"
                elif not higher_better and diff < 0:
                    str_row += " & " + "{\\color[HTML]{" +  get_color(math.fabs(diff), 0 , math.fabs(max_value), min_positive_col, max_positive_col,) + "}" f" {diff:.2f} " + "}"
                else:
                    str_row += " & " + f"{diff:.2f}"
    else:
        for i, model in enumerate(models):
            if i % 2 == 0:
                base_model = model
            if model == base_model:
                str_row += " & " + f"{row[model]:.2f}"
            else:
                diff = row[model] - row[base_model]
                str_row += " & " + f"{diff:.2f}"

    return str_row +  " \\\\ \n"

def prepare_caption(individual_results):
    ret_str = "Number of Played Episodes per model: \n"
    for model, dff in individual_results.items():
        ret_str += f"{model.replace("_", "\_")}: {len(dff)}; "

    return ret_str



def print_latex_full_table_metrics(data: pd.DataFrame, metrics_config: dict, min_value: int, max_value: int, success_only_cols: list, metric_alias: dict = {}, sub_headers=None, caption: str = "", label:str = "", min_positive_col: str = "", max_positive_col:str = "", min_negative_color: str= "", max_negative_color:str = ""):
    if sub_headers is None:
        sub_headers = {}

    models = [c for c in data.columns if c != "metric"]

    header: str = """
\\begin{table}[h!]
\\resizebox{\\textwidth}{!}{%
\\begin{tabular}{lrrrrrr}
\\hline
                                 & \\multicolumn{2}{c}{\\textbf{Llama 8B}} & \\multicolumn{2}{c}{\\textbf{Mistral-Small}} & \\multicolumn{2}{c}{\\textbf{Llama 70B}}\\\\
\\textbf{Metric}                 & \\textbf{Baseline} & \\textbf{Fine-Tuned} & \\textbf{Baseline} & \\textbf{Fine-Tuned} & \\textbf{Baseline} & \\textbf{Fine-Tuned} \\\\ \\hline
    """

    footer = """
\\hline
\\end{tabular}%
}
\\caption{""" + caption + """}
\\label{tab:""" + label + """}
\\end{table}"""

    body = ""
    for i, row in data.iterrows():
        metric = row["metric"]
        if metric in sub_headers.keys():
            sub_header = "\\\\ \\multicolumn{2}{l}{\\textbf{{" + f"{sub_headers[metric]}" + "}}}" + " &&&& \\\\ \n"
            body += sub_header
        str_values =  prepare_row(
            row=row,
            models=models,
            metric_config=metrics_config,
            min_value=min_value,
            max_value=max_value,
            min_positive_col=min_positive_col,
            max_positive_col=max_positive_col,
            min_negative_color=min_negative_color,
            max_negative_color=max_negative_color,
            success_only_cols=success_only_cols,
            metric_alias=metric_alias)

        body += str_values
    return header + body + footer##
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
    return '{:02X}{:02X}{:02X}'.format(rgb[0], rgb[1], rgb[2])

def get_color(value, min_val, max_val, min_color, max_color):
    # Clamp value within range
    clamped = max(min(value, max_val), min_val)

    # Calculate interpolation ratio
    if max_val == min_val:
        ratio = 0.5
    else:
        ratio = (clamped - min_val) / (max_val - min_val)

    # Convert colors to RGB
    min_rgb = hex_to_rgb(min_color)
    max_rgb = hex_to_rgb(max_color)

    # Interpolate RGB values
    interpolated = [
        int(min_rgb[i] + ratio * (max_rgb[i] - min_rgb[i]))
        for i in range(3)
    ]

    return rgb_to_hex(interpolated)


games = ["wordle", "textmapworld", "matchit_ascii", "adventuregame", "codenames", "imagegame", "privateshared", "referencegame", "taboo", "wordle", "wordle_withclue", "wordle_withcritic",  "textmapworld_specificroom", "textmapworld_graphreasoning", "guesswhat"]
for game in games:
    print(game)
    metric_order = metric_orders[game]
    metrics_config = metrics_configs[game]
    sub_headers = all_sub_headers[game]
    all_success_metrics = success_only_cols[game]
    df, individual_results = prepare_result_df(game, metric_order, all_success_metrics, base_metrics=base_metrics)
    df_diff_to_baseline = df.copy()



    # get the differences for each model
    for key in model_lookup.keys():
        base = key
        tuned = model_lookup[key]

        df_diff_to_baseline[tuned] = df_diff_to_baseline[tuned] - df_diff_to_baseline[base]
        #df_diff_to_baseline[tuned] = df_diff_to_baseline[tuned].apply(lambda x: x * -1)

    # reduce to fine tuned difference
    df_diff_to_baseline = df_diff_to_baseline[list(model_lookup.values()) + ['metric']]
    higher_better_columns = [key for key in metrics_config if metrics_config[key]]
    lower_better_columns = [key for key in metrics_config if not metrics_config[key]]

    higher_better_df = df_diff_to_baseline[df_diff_to_baseline.metric.isin(higher_better_columns)].drop(["metric"], axis=1)
    lower_better_df = df_diff_to_baseline[df_diff_to_baseline.metric.isin(lower_better_columns)].drop(["metric"], axis=1)

    all_max = (higher_better_df.max().max(), math.fabs(lower_better_df.min().min()))
    all_min = (higher_better_df.min().min(), lower_better_df.max().max() * -1,)

    # get min max for the whole df
    max_val = max(all_max)
    min_val = min(all_min)

    caption: str = f"Individual Game Metrics for {game.capitalize().replace("_", "\_")}; \\\\ " + prepare_caption(individual_results)
    label: str = f"IndividualMetrics{game}"

    latex_string = print_latex_full_table_metrics(
        data=df,
        metrics_config=metrics_config,
        success_only_cols=all_success_metrics,
        sub_headers=sub_headers,
        caption=caption,
        label=label,
        max_value=max_val,
        min_value=min_val,
        min_positive_col="023020", #"90EE90"
        max_positive_col="90EE90",
        min_negative_color="8B0000", #FF0000
        max_negative_color="FF0000",
        metric_alias=general_column_alias
    )

    save_path = f'../../latex_tables/game_metrics/{game}.tex'
    with open(save_path, 'w') as file:
        file.write(latex_string)



<>:192: SyntaxWarning: invalid escape sequence '\_'
<>:307: SyntaxWarning: invalid escape sequence '\_'
<>:192: SyntaxWarning: invalid escape sequence '\_'
<>:307: SyntaxWarning: invalid escape sequence '\_'
/var/folders/rp/55m2cs9j4nq0zsmq1sdq68kw0000gn/T/ipykernel_38973/3742608473.py:192: SyntaxWarning: invalid escape sequence '\_'
  ret_str += f"{model.replace("_", "\_")}: {len(dff)}; "
/var/folders/rp/55m2cs9j4nq0zsmq1sdq68kw0000gn/T/ipykernel_38973/3742608473.py:307: SyntaxWarning: invalid escape sequence '\_'
  caption: str = f"Individual Game Metrics for {game.capitalize().replace("_", "\_")}; \\\\ " + prepare_caption(individual_results)


wordle
Index(['game', 'model', 'experiment', 'episode', 'Aborted', 'Lose',
       'Main Score', 'Parsed Request Count', 'Played', 'Request Count',
       'Request Success Ratio', 'Success', 'Violated Request Count',
       'repeats guess', 'total guess repetitions'],
      dtype='string', name='metric')
[]
[]
[]
[]
[]
[]
[]
textmapworld
Index(['game', 'model', 'experiment', 'episode', 'Aborted', 'Lose',
       'Main Score', 'Played', 'Success', 'efficiency', 'exploration',
       'invalid_moves', 'loops', 'moves', 'number_visited', 'seen', 'stopped',
       'turns_limit', 'valid_moves'],
      dtype='string', name='metric')
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
matchit_ascii
Index(['game', 'model', 'experiment', 'episode', 'Aborted', 'Lose',
       'Main Score', 'Parsed Request Count', 'Played', 'Player Score',
       'Request Count', 'Success', 'Violated Request Count'],
      dtype='string', name='metric')
[]
[]
[]
[]
[]
[]
adventuregame
Index(['game', 'model', 'experiment', 'episode', 'A

In [47]:
print(prepare_caption(individual_results))

Num Played Episodes per model: 
llama_8B_base: 40; llama_8B_fine: 39; mistral_base: 40; mistral_fine: 40; llama_70B_base: 39; llama_70B_fine: 40; 


In [16]:
current_game = "imagegame"

df_mistral_raw = load_raw_csv("mistral" ,"", "", current_game)
df_llama8B_raw = load_raw_csv("llama_8B" ,"", "", current_game)
df_llama70B_raw = load_raw_csv("llama_70B"  ,"", "", current_game)


for col in df_llama8B_raw.columns:
    if col not in std_cols:
        df_mistral_raw[col] = pd.to_numeric(df_mistral_raw[col], errors='coerce')
        df_llama8B_raw[col] = pd.to_numeric(df_llama8B_raw[col], errors='coerce')
        df_llama70B_raw[col] = pd.to_numeric(df_llama70B_raw[col], errors='coerce')


df_l8_base = df_llama8B_raw[df_llama8B_raw.model == base_model_llama_8B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
df_l8_base_all_aborted = df_l8_base[df_l8_base.Aborted == 1]
df_l8_base_all_lost = df_l8_base[df_l8_base.Lose == 1]
df_l8_base_all_success = df_l8_base[df_l8_base.Success == 1]

df_l8_final = df_llama8B_raw[df_llama8B_raw.model == final_model_llama_8B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
df_l8_fine_all_aborted = df_l8_final[df_l8_final.Aborted == 1]
df_l8_fine_all_lost = df_l8_final[df_l8_final.Lose == 1]
df_l8_fine_all_success = df_l8_final[df_l8_final.Success == 1]

df_l70_base = df_llama70B_raw[df_llama70B_raw.model == base_model_llama_70B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
df_l70_base_all_aborted = df_l70_base[df_l70_base.Aborted == 1]
df_l70_base_all_lost = df_l70_base[df_l70_base.Lose == 1]
df_l70_base_all_success = df_l70_base[df_l70_base.Success == 1]

df_l70_final = df_llama70B_raw[df_llama70B_raw.model == final_model_llama_70B].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
df_l70_fine_all_aborted = df_l70_final[df_l70_final.Aborted == 1]
df_l70_fine_all_lost = df_l70_final[df_l70_final.Lose == 1]
df_l70_fine_all_success = df_l70_final[df_l70_final.Success == 1]

df_mistral_base = df_mistral_raw[df_mistral_raw.model == base_model_mistral].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
df_mistral_base_all_aborted = df_mistral_base[df_mistral_base.Aborted == 1]
df_mistral_base_all_lost = df_mistral_base[df_mistral_base.Lose == 1]
df_mistral_base_all_success = df_mistral_base[df_mistral_base.Success == 1]

df_mistral_final = df_mistral_raw[df_mistral_raw.model == final_model_mistral].drop(['game', 'model', 'episode', 'experiment'], axis=1).fillna(0)
df_mistral_fine_all_aborted = df_mistral_final[df_mistral_final.Aborted == 1]
df_mistral_fine_all_lost = df_mistral_final[df_mistral_final.Lose == 1]
df_mistral_fine_all_success = df_mistral_final[df_mistral_final.Success == 1]

In [9]:
len(df_l70_base_all_aborted), len(df_l70_base_all_lost), len(df_l70_base_all_success)

(15, 66, 47)

In [10]:
len(df_l70_fine_all_aborted), len(df_l70_fine_all_lost), len(df_l70_fine_all_success)

(29, 48, 51)

In [17]:
curr_model = base_model_mistral
model_file = "mistral"
curr_data = df_mistral_raw[df_mistral_raw.model == curr_model]
current_aborted = curr_data[curr_data.Aborted == 1]


transcript_html_filenames = []

for i, row in current_aborted.iterrows():
    html_dir = base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"] + '/transcript.html'
    transcript_html_filenames.append(html_dir)

len(transcript_html_filenames)

53

In [18]:
import webbrowser

# open all transcripts
new = 2
for html in transcript_html_filenames:
    url = f"file://{html}"
    print(url)
    webbrowser.open(url,new=new)

file:///Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/mistral/Unsloth-Mistral-Small-24B-Instruct-2501-t0.0--Unsloth-Mistral-Small-24B-Instruct-2501-t0.0/imagegame/0_compact_grids/episode_0/transcript.html
file:///Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/mistral/Unsloth-Mistral-Small-24B-Instruct-2501-t0.0--Unsloth-Mistral-Small-24B-Instruct-2501-t0.0/imagegame/0_compact_grids/episode_10/transcript.html
file:///Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/mistral/Unsloth-Mistral-Small-24B-Instruct-2501-t0.0--Unsloth-Mistral-Small-24B-Instruct-2501-t0.0/imagegame/0_compact_grids/episode_11/transcript.html
file:///Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/mistral/Unsloth-Mistral-Small-24B-Instruct-2501-t0.0--Unsloth-Mistral-Small-24B-Instruct-2501-t0.0/imagegame/0_compact_grids/

In [160]:
import json

def get_errors_codenames():
    errors = {}
    for i, row in current_aborted.iterrows():
        html_dir = base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"] + '/interactions.json'
        data = json.load(open(html_dir))
        tp = data["turns"][-1][-1]["action"]["type"]
        player =  data["turns"][-1][-1]["action"]["content"]["player"]
        err_tpye =  data["turns"][-1][-1]["action"]["content"]["type"]

        if err_tpye == "clue contains spaces":
            print(base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"])

        if player not in errors.keys():
            errors[player] = {}

        if err_tpye in  errors[player]:
            errors[player][err_tpye] += 1
        else:
            errors[player][err_tpye] = 1
    return errors

def get_errors_adventuregame():
    errors = {}
    for i, row in current_aborted.iterrows():
        html_dir = base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"] + '/interactions.json'
        data = json.load(open(html_dir))
        tp = data["turns"][-1][-2]["action"]["type"]
        err_tpye = data["turns"][-1][-2]["action"]["content"]

        if tp not in errors.keys():
            errors[tp] = {}

        if err_tpye in  errors[tp]:
            errors[tp][err_tpye] += 1
        else:
            errors[tp][err_tpye] = 1
    return errors

def get_errors_wordle():
    errors = {}
    for i, row in current_aborted.iterrows():
        html_dir = base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"] + '/interactions.json'
        data = json.load(open(html_dir))
        tp = data["turns"][-1][-1]["action"]["type"]
        err_tpye = data["turns"][-1][-2]["action"]["content"]

        if tp not in errors.keys():
            errors[tp] = {}

        if err_tpye in  errors[tp]:
            errors[tp][err_tpye] += 1
        else:
            errors[tp][err_tpye] = 1
    return errors

def get_errors_wordle_withclue():
    errors = {}
    for i, row in current_aborted.iterrows():
        html_dir = base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"] + '/interactions.json'
        data = json.load(open(html_dir))
        tp = data["turns"][-1][-1]["action"]["type"]
        err_tpye = data["turns"][-1][-2]["action"]["content"]

        if tp not in errors.keys():
            errors[tp] = {}

        if err_tpye in  errors[tp]:
            errors[tp][err_tpye] += 1
        else:
            errors[tp][err_tpye] = 1
    return errors

def get_errors_wordle_withcritic():
    errors = {}
    for i, row in current_aborted.iterrows():
        html_dir = base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"] + '/interactions.json'
        data = json.load(open(html_dir))
        tp = data["turns"][-1][-1]["action"]["type"]
        err_tpye = data["turns"][-1][-2]["action"]["content"]

        if tp not in errors.keys():
            errors[tp] = {}

        if err_tpye in  errors[tp]:
            errors[tp][err_tpye] += 1
        else:
            errors[tp][err_tpye] = 1
    return errors

def get_errors_guess_what():
    errors = {}
    for i, row in current_aborted.iterrows():
        html_dir = base_path_results + '/' +  model_file  + '/' +   curr_model + '/' + row["game"] + '/' + row["experiment"] + '/' + row["episode"] + '/interactions.json'
        data = json.load(open(html_dir))
        tp = data["turns"][-1][-1]["action"]["type"]
        err_tpye = data["turns"][-1][-2]["action"]["content"]

        if tp not in errors.keys():
            errors[tp] = {}

        if err_tpye in  errors[tp]:
            errors[tp][err_tpye] += 1
        else:
            errors[tp][err_tpye] = 1
    return errors

## manual games
['matchit_ascii', "all tmw", "imagegame", "privateshared", "taboo"]

get_errors_wordle_withclue()

{'invalid format': {'Guess does not conform to the format rules\nError: The length of the guessed word is not 5.': 8,
  "Guess does not conform to the format rules\nError: The response should contain the 'guess:' keyword only once.": 6,
  "Guess does not conform to the format rules\nError: The response should contain only the 'guess:' and 'explanation:' keywords and associated information.": 1}}

In [125]:
target_episodes = [
                    "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/llama_8B/llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0/codenames/11_concrete/episode_3",
                    "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/llama_8B/llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0/codenames/12_abstract/episode_2",
                    "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/llama_8B/llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0/codenames/2_easy/episode_0",
                    "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/llama_8B/llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0/codenames/3_difficult/episode_5",
                    "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/llama_8B/llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0/codenames/4_none/episode_9",
                    "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/llama_8B/llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0/codenames/8_high/episode_2",
                    "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/benchmark_results/clembench/final_models/llama_8B/llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0--llama3.1-8B-sft-e1-DFINAL_0.7K-steps-t0.0/codenames/9_unambiguous/episode_4"
]

res_data = {

}
for t in target_episodes:
    path = t + "/instance.json"
    data = json.load(open(path))
    board = data["board"]
    targets = data["assignments"]["team"]
    res_data[t] = {
        "board": board,
        "targets": targets
    }

In [111]:
taboo_new_path = "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/data/raw/taboo_new_raw.jsonl"
taboo_old_path = "/Users/nicolahorst/Documents/GitHub/clem-project_playpen/data/raw/taboo_new_raw.jsonl"

df_new_taboo = pd.read_json(taboo_new_path, lines=True)
df_old_taboo = pd.read_json(taboo_old_path, lines=True)
all_taboo_targets = list(set(list(df_old_taboo.target_word) + list(df_new_taboo.target_word)))

In [116]:
for key in res_data.keys():
    board = res_data[key]["board"]
    targets = res_data[key]["targets"]
    # check whether a board word is in the taboo targets
    intersection_board = [w for w in targets if w in all_taboo_targets]

    if len(intersection_board) > 0:
        print(intersection_board)

In [112]:
all_taboo_targets.sort()
all_taboo_targets

['anymore',
 'array',
 'assure',
 'autograph',
 'boy',
 'bypass',
 'cabinet',
 'career',
 'clear',
 'combination',
 'conform',
 'contributor',
 'deduction',
 'designation',
 'embroidery',
 'envelope',
 'fat',
 'fighter',
 'filter',
 'harmony',
 'haven',
 'host',
 'implication',
 'independently',
 'induce',
 'invasion',
 'likely',
 'limousine',
 'myth',
 'nil',
 'none',
 'obvious',
 'orient',
 'output',
 'passage',
 'pastor',
 'peaceful',
 'planner',
 'plaza',
 'principal',
 'provide',
 'quarterly',
 'rate',
 'recorder',
 'regret',
 'rental',
 'responsibility',
 'resume',
 'scoop',
 'sear',
 'seize',
 'shark',
 'short',
 'squeeze',
 'stimulate',
 'subsidiary',
 'synthesis',
 'touch',
 'transit',
 'transplant']

## Errors Codenames
### LLama 70B base:
{'Guesser': {'guess is clue word': 13, 'guess word is hallucination': 4}, 'Cluegiver': {'target is hallucination': 2, 'target was already guessed': 1}}
### LLama 70B fine:
{'Guesser': {'guess is clue word': 12, 'wrong number of guesses': 2, 'guess word is hallucination': 1}, 'Cluegiver': {'target is hallucination': 4, 'clue is morphologically related to word on the board': 6, 'target was already guessed': 1}}

### llama 8B base
{'Guesser': {'guess is clue word': 39, 'guess appears more than once in utterance': 1, 'guess word is hallucination': 12, 'wrong number of guesses': 11, 'guess was already guessed': 1}, 'Cluegiver': {'target is hallucination': 6, 'clue is morphologically related to word on the board': 1, 'target was already guessed': 3}}
### llama 8B fine
{'Guesser': {'guess is clue word': 40, 'guess word is hallucination': 8, 'wrong number of guesses': 1, 'guess was already guessed': 4}, 'Cluegiver': {'clue is morphologically related to word on the board': 25, 'target was already guessed': 6, 'target is hallucination': 6, 'clue contains spaces': 7}}

### mistral base
{'Guesser': {'prefix error': 4, 'guess is clue word': 25, 'guess word is hallucination': 1, 'wrong number of guesses': 2}, 'Cluegiver': {'clue is morphologically related to word on the board': 6}}
prefix error in guesser means that it did not add GUESS:

### mistral fine
{'Guesser': {'guess is clue word': 14, 'wrong number of guesses': 1}, 'Cluegiver': {'clue is morphologically related to word on the board': 24, 'clue contains spaces': 2, 'target was already guessed': 1}}


In [12]:
ddd = individual_results["llama_8B_fine"]
dfd = ddd[ddd.Aborted != 1]

In [13]:
dfd["Main Score"].sum()/len(dfd["Main Score"])

np.float64(1.1764705882352942)

In [ ]:
dfd["Main Score"].sum()

In [11]:
individual_results.keys()

dict_keys(['llama_8B_base', 'llama_8B_fine', 'mistral_base', 'mistral_fine', 'llama_70B_base', 'llama_70B_fine'])